In [46]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from fast_ml.model_development import train_valid_test_split
from sklearn.preprocessing import LabelEncoder

In [32]:
import dvc.api

with dvc.api.open(
    'data/data_with_os.csv',
    mode='rb',
    
) as data:
    df_with_os = pd.read_csv(data)
    df_with_os.head()
df_with_os.drop(['auction_id'],axis=1,inplace=True)

In [38]:
df_with_os['platform_os']=df_with_os['platform_os'].astype('category')
df_with_os['user_response']=df_with_os['user_response'].astype('category')

In [41]:
categorical_column = df_with_os.select_dtypes(include=["object","category"]).columns.tolist()

# Get column names have less than 10 more than 2 unique values
to_one_hot_encoding = [col for col in categorical_column if df_with_os[col].nunique() <= 10 and df_with_os[col].nunique() > 2]

# Get Categorical Column names thoose are not in "to_one_hot_encoding"
to_label_encoding = [col for col in categorical_column if not col in to_one_hot_encoding]

print("To One Hot Encoding:", to_one_hot_encoding)
print("To Label Encoding:", to_label_encoding)

To One Hot Encoding: ['date']
To Label Encoding: ['experiment', 'device_make', 'platform_os', 'user_response']


In [44]:
one_hot_encoded_columns = pd.get_dummies(df_with_os[to_one_hot_encoding])
one_hot_encoded_columns

,date_2020-07-03,date_2020-07-04,date_2020-07-05,date_2020-07-06,date_2020-07-07,date_2020-07-08,date_2020-07-09,date_2020-07-10
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1238,0,0,0,0,0,1,0,0
1239,0,0,0,0,0,0,1,0
1240,0,0,0,0,0,0,1,0
1241,0,0,0,0,0,0,0,1


In [48]:
# Label Encoding

label_encoded_columns = []
# For loop for each columns
for col in to_label_encoding:
    # We define new label encoder to each new column
    le = LabelEncoder()
    # Encode our data and create new Dataframe of it, 
    # notice that we gave column name in "columns" arguments
    column_dataframe = pd.DataFrame(le.fit_transform(df_with_os[col]), columns=[col] )
    # and add new DataFrame to "label_encoded_columns" list
    label_encoded_columns.append(column_dataframe)

# Merge all data frames
label_encoded_columns = pd.concat(label_encoded_columns, axis=1)
label_encoded_columns

,experiment,device_make,platform_os,user_response
0,1,13,1,1
1,0,43,1,1
2,0,13,1,1
3,1,65,1,1
4,1,13,1,1
...,...,...,...,...
1238,1,69,1,0
1239,1,13,1,0
1240,0,13,1,0
1241,0,13,1,0


In [29]:
df_with_os['hour'].value_counts()

15    281
8      67
7      59
10     57
9      57
14     51
6      50
5      49
4      46
20     45
11     44
3      44
16     44
12     42
13     41
18     38
19     38
17     34
21     34
2      33
1      32
0      31
22     21
23      5
Name: hour, dtype: int64

In [22]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df_with_os, target = 'platform_os', 
                                                                            train_size=0.7, valid_size=0.2, test_size=0.1)
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


(None, None)

In [47]:
# Define Random Forest Model
rf = RandomForestClassifier(n_estimators=100)

# We fit our model with our train data
rf.fit(X_train, y_train)

# Then predict results from X_test data
pred_rf = rf.predict(X_test)

# See First 10 Predictions and They Actual Values
print("Predicted:", pred_rf[0:10])
print("Actual:", y_test[0:10])

ValueError: could not convert string to float: 'exposed'